In [135]:
import pandas as pd
import os

In [136]:
path = os.path.join('OriginalData', 'education.csv')
data = pd.read_csv(path)
data_backup = data.copy()

In [137]:
data = data.head(1000)

In [138]:
# Lowercase all rows in the dataframe
data = data.applymap(lambda s:s.lower() if type(s) == str else s)
data

,user_id,school_name,degree,fields_of_study,start_year_month,end_year_month
0,0,anadolu üniversitesi,NaN,NaN,NaN,NaN
1,0,adıyaman üniversitesi,NaN,NaN,NaN,NaN
2,0,fırat üniversitesi,NaN,NaN,NaN,NaN
3,1,fırat üniversitesi,yüksek lisans,yazılım mühendisliği,NaN,NaN
4,1,fırat üniversitesi,lisans,yazılım mühendisliği,NaN,NaN
...,...,...,...,...,...,...
995,587,uniwersytet łódzki,bachelor's degree,bilgisayar mühendisliği,NaN,NaN
996,588,fırat üniversitesi,NaN,NaN,NaN,NaN
997,588,universitatea „petru maior” din târgu-mureș,lisans derecesi,bilgisayar bilimleri,NaN,NaN
998,589,fırat üniversitesi,lisans derecesi,adli bilişim mühendisliği,NaN,NaN


In [139]:
# add a column for each user, with the value of the amount of rows
# that the user has in the dataset
data['education_count'] = data.groupby('user_id')['user_id'].transform('count')

In [140]:
# remove trailing and leading spaces
data['fields_of_study'] = data['fields_of_study'].str.strip()

In [141]:
# Has went to university
data['education_university'] = data['school_name'].apply(lambda x: 1 if 'üniversite' in x or 'university' in x or 'uni' in x or 'üni' in x or 'univ' in x else 0)
data

,user_id,school_name,degree,fields_of_study,start_year_month,end_year_month,education_count,education_university
0,0,anadolu üniversitesi,NaN,NaN,NaN,NaN,3,1
1,0,adıyaman üniversitesi,NaN,NaN,NaN,NaN,3,1
2,0,fırat üniversitesi,NaN,NaN,NaN,NaN,3,1
3,1,fırat üniversitesi,yüksek lisans,yazılım mühendisliği,NaN,NaN,2,1
4,1,fırat üniversitesi,lisans,yazılım mühendisliği,NaN,NaN,2,1
...,...,...,...,...,...,...,...,...
995,587,uniwersytet łódzki,bachelor's degree,bilgisayar mühendisliği,NaN,NaN,2,1
996,588,fırat üniversitesi,NaN,NaN,NaN,NaN,2,1
997,588,universitatea „petru maior” din târgu-mureș,lisans derecesi,bilgisayar bilimleri,NaN,NaN,2,1
998,589,fırat üniversitesi,lisans derecesi,adli bilişim mühendisliği,NaN,NaN,1,1


In [142]:
from fuzzywuzzy import fuzz
from collections import defaultdict

def group_strings(strings, reference_strings, similarity_threshold):
    groups = defaultdict(list)
    for i, string1 in enumerate(reference_strings):
        for j, string2 in enumerate(strings):
            similarity = fuzz.token_set_ratio(string1, string2)
            if similarity >= similarity_threshold:
                groups[string1].append(string2)
    return groups

In [143]:
# Engineering, IT, science
engineering_groups = ["Mühendis", "Mühendislik", "Müh.", "Mühendisliği", "Engineering", "Engi", "Bilgisayar", "Bilişim", "Bilim", "Teknoloji", "Meka", "Makine", "Machine", "Veri"]

# Business, Marketing etc.
business_group = ["Finance", "Marketing", "Business", "İş", "Pazarlama", "İşletme", ]

# Lower all the groups
engineering_groups = [x.lower() for x in engineering_groups]
business_group = [x.lower() for x in business_group]


skill_groups = engineering_groups + business_group
skill_groups

['mühendis',
 'mühendislik',
 'müh.',
 'mühendisliği',
 'engineering',
 'engi',
 'bilgisayar',
 'bilişim',
 'bilim',
 'teknoloji',
 'meka',
 'makine',
 'machine',
 'veri',
 'finance',
 'marketing',
 'business',
 'i̇ş',
 'pazarlama',
 'i̇şletme']

In [144]:
skills = data['fields_of_study'].unique().tolist()

In [145]:
skill_groups = group_strings(skills, skill_groups, 50)

In [146]:
# Combine it skills
engineering_fields = []
for key, value in skill_groups.items():
    if key in engineering_groups:
        engineering_fields += value
it_skills = list(set(engineering_fields))

# Combine business skills
business_fields = []
for key, value in skill_groups.items():
    if key in business_group:
        business_fields += value
business_skills = list(set(business_fields))
